In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_squared_error

In [2]:
# Load the datasets
X = np.load("Xtrain_Regression_Part2.npy")
X_TESTE = np.load("Xtest_Regression_Part2.npy")
Y = np.load("Ytrain_Regression_Part2.npy")

CÓDIGO PARA ENTREGAR

In [3]:
# model = linear_model.Lasso(alpha=0.002)
model = linear_model.LinearRegression()
# model = linear_model.Ridge(alpha=0.002)


score = 10

X_for = X
Y_for = Y
removed = 0

# train and predict
model.fit(X_for, Y_for)

Y_pred = model.predict(X_for)

diff = (Y_for - Y_pred.reshape(Y_for.shape[0], 1))**2
avg_diff = np.average(diff)

print(f"avg distance is {avg_diff}")

while removed < 9:
    max_diff = max(diff)

    if max_diff > avg_diff:
        index = np.where(diff == max_diff)

        print(f"Index {index[0][0]}: element {Y_for[index]} is outlier, dist = {diff[index]}")

        Y_for = np.delete(Y_for, index, 0)
        X_for = np.delete(X_for, index, 0)
        diff = np.delete(diff, index, 0)

        removed += 1
    
    else:
        break

# train and predict
model.fit(X_for, Y_for)

Y_pred = model.predict(X_for)

current_score = -1 * np.average(
    cross_val_score(model, X_for, Y_for, scoring="neg_mean_squared_error", cv=5)
)
score = min(score, current_score)

print(f"best is {score}")


avg distance is 1.3241159445831923
Index 85: element [-7.15909879] is outlier, dist = [79.00975928]
Index 77: element [4.75365081] is outlier, dist = [26.40160169]
Index 92: element [-3.46827343] is outlier, dist = [2.33656163]
Index 33: element [1.08273097] is outlier, dist = [1.97456985]
Index 89: element [-0.02393726] is outlier, dist = [1.41963421]
best is 0.016760745347899328
